## Project name: Detecting exoplanets in outer space
#### 1. Dataset: https://www.kaggle.com/keplersmachines/kepler-labelled-time-series-data/ 
#### 2. Model: Tensorflow Boosted Trees Model

In [5]:
import tensorflow as tf
import numpy as np
print('tensorflow version: ',tf.__version__)
import os
import sys
print('python interpreter and version: ', sys.version)

tensorflow version:  1.13.1
python interpreter and version:  3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]


### Load kepler data

In [14]:
dsroot = 'C:\\Users\\peiya\\Desktop\\Quan\\detecting_exoplanets_in_outer_space';
os.listdir(dsroot)

['.ipynb_checkpoints',
 'detecting_exoplanets_in_outer_space.ipynb',
 'exoTest.csv',
 'exoTrain.csv']

In [15]:
import pandas as pd
train = pd.read_csv(os.path.join(dsroot, 'exoTrain.csv'))
test = pd.read_csv(os.path.join(dsroot, 'exoTest.csv'))
print('training data \n', train.head())
print('testing data \n', test.head())


training data 
    LABEL   FLUX.1   FLUX.2   FLUX.3   FLUX.4   FLUX.5   FLUX.6  FLUX.7  \
0      2    93.85    83.81    20.10   -26.98   -39.56  -124.71 -135.18   
1      2   -38.88   -33.83   -58.54   -40.09   -79.31   -72.81  -86.55   
2      2   532.64   535.92   513.73   496.92   456.45   466.00  464.50   
3      2   326.52   347.39   302.35   298.13   317.74   312.70  322.33   
4      2 -1107.21 -1112.59 -1118.95 -1095.10 -1057.55 -1034.48 -998.34   

    FLUX.8  FLUX.9  ...  FLUX.3188  FLUX.3189  FLUX.3190  FLUX.3191  \
0   -96.27  -79.89  ...     -78.07    -102.15    -102.15      25.13   
1   -85.33  -83.97  ...      -3.28     -32.21     -32.21     -24.89   
2   486.39  436.56  ...     -71.69      13.31      13.31     -29.89   
3   311.31  312.42  ...       5.71      -3.73      -3.73      30.05   
4 -1022.71 -989.57  ...    -594.37    -401.66    -401.66    -357.24   

   FLUX.3192  FLUX.3193  FLUX.3194  FLUX.3195  FLUX.3196  FLUX.3197  
0      48.57      92.54      39.32      61

In [17]:
x_train = train.drop('LABEL', axis=1)
y_train = train.LABEL-1 # TFBT estimator assumes labels starting with 0, in the datasets are 1 and 2 instead 
x_test = test.drop('LABEL', axis=1)
y_test = test.LABEL-1

### Tensorflow boosted trees model

In [18]:
numeric_col_headers = x_train.columns.values.tolist() # save names of all features in a vector

bc_fn = tf.feature_column.bucketized_column
nc_fn = tf.feature_column.numeric_column

buketized_features = [bc_fn(source_column=nc_fn(key=column),
                           boundaries=[x_train[column].mean()])
                     for column in numeric_col_headers]
all_features = buketized_features

In [23]:
batch_size = 32

pi_fn = tf.estimator.inputs.pandas_input_fn
train_input_fn = pi_fn(x=x_train,
                      y=y_train,
                      batch_size=batch_size,
                      shuffle=True,
                      num_epochs=None)

eval_input_fn = pi_fn(x=x_test,
                      y=y_test,
                      batch_size=batch_size,
                      shuffle=False,
                      num_epochs=1)


In [24]:
n_trees = 100
n_steps = 100

m_fn = tf.estimator.BoostedTreesClassifier
model = m_fn(feature_columns=all_features,
            n_trees=n_trees,
            n_batches_per_layer=batch_size,
            model_dir='./tfbtmodel')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tfbtmodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001EDC3554EB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
model.train(input_fn=train_input_fn, steps=n_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./tfbtmodel\model.ckpt-18
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving checkpoints for 18 into ./tfbtmodel\model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:loss = 0.6931472, step = 19
INFO:tensorflow:Saving checkpoints for 118 into ./tfbtmod

In [26]:
results = model.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-16T12:02:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tfbtmodel\model.ckpt-118
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-16-12:02:42
INFO:tensorflow:Saving dict for global step 118: accuracy = 0.99122804, accuracy_baseline = 0.99122804, auc = 0.6925664, auc_precision_recall = 0.03408982, average_loss = 0.46484697, global_step = 118, label/mean = 0.00877193, loss = 0.46477324, precision = 0.0, prediction/mean = 0.36888078, recall = 0.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for

In [27]:
for key, value in sorted(results.items()):
    print('{}: {}'.format(key, value))

accuracy: 0.9912280440330505
accuracy_baseline: 0.9912280440330505
auc: 0.6925663948059082
auc_precision_recall: 0.034089818596839905
average_loss: 0.46484696865081787
global_step: 118
label/mean: 0.008771929889917374
loss: 0.4647732377052307
precision: 0.0
prediction/mean: 0.3688807785511017
recall: 0.0
